In [1]:
#!/usr/bin/env python3
"""
Demo script for Town04 Figure-8 track navigation in CARLA.
This script demonstrates track analysis and autonomous vehicle control.
"""

import carla
import time
import random
from typing import Optional
import sys

def connect_to_carla(host: str = 'localhost', port: int = 2000, 
                    timeout: int = 10) -> Optional[carla.Client]:
    """
    Connect to CARLA server.
    
    Args:
        host: Server hostname
        port: Server port
        timeout: Connection timeout in seconds
        
    Returns:
        CARLA client instance or None if connection fails
    """
    try:
        client = carla.Client(host, port)
        client.set_timeout(timeout)
        return client
    except Exception as e:
        print(f"Failed to connect to CARLA server: {e}")
        return None

def setup_world(client: carla.Client) -> Optional[carla.World]:
    """
    Load and setup Town04 world.
    
    Args:
        client: Connected CARLA client
        
    Returns:
        CARLA world instance or None if setup fails
    """
    try:
        world = client.load_world('Town04')
        settings = world.get_settings()
        settings.synchronous_mode = True
        settings.fixed_delta_seconds = 0.05
        world.apply_settings(settings)
        return world
    except Exception as e:
        print(f"Failed to setup world: {e}")
        return None

def spawn_vehicle(world: carla.World, spawn_point: Optional[carla.Transform] = None) -> Optional[carla.Vehicle]:
    """
    Spawn a vehicle in the world.
    
    Args:
        world: CARLA world instance
        spawn_point: Optional specific spawn point
        
    Returns:
        Spawned vehicle instance or None if spawn fails
    """
    try:
        blueprint_library = world.get_blueprint_library()
        vehicle_bp = random.choice(blueprint_library.filter('vehicle.tesla.model3'))
        
        if spawn_point is None:
            spawn_points = world.get_map().get_spawn_points()
            spawn_point = random.choice(spawn_points)
        
        vehicle = world.spawn_actor(vehicle_bp, spawn_point)
        return vehicle
    except Exception as e:
        print(f"Failed to spawn vehicle: {e}")
        return None

def setup_spectator(world: carla.World, vehicle: carla.Vehicle):
    """
    Position spectator to view the vehicle.
    
    Args:
        world: CARLA world instance
        vehicle: Vehicle to follow
    """
    spectator = world.get_spectator()
    transform = vehicle.get_transform()
    spectator.set_transform(carla.Transform(
        transform.location + carla.Location(z=50),
        carla.Rotation(pitch=-90)
    ))

def main():
    """Main function to run the demo."""
    # Connect to CARLA
    client = connect_to_carla()
    if client is None:
        return
    
    # Setup world
    world = setup_world(client)
    if world is None:
        return
    
    try:
        # Create Figure-8 track manager
        track_manager = Town04Figure8(world)
        
        # Analyze track
        print("\nAnalyzing track...")
        eight_lane_roads = track_manager.analyze_eight_lane_roads()
        
        # Validate waypoints
        print("\nValidating waypoints...")
        road_waypoints = track_manager.validate_figure8_waypoints()
        
        # Spawn vehicle
        vehicle = spawn_vehicle(world)
        if vehicle is None:
            return
        
        # Setup spectator
        setup_spectator(world, vehicle)
        
        # Create vehicle controller
        controller = VehicleController(world, vehicle)
        
        print("\nStarting autonomous navigation...")
        print("Press Ctrl+C to stop")
        
        # Start autonomous navigation
        controller.drive_figure_eight(road_waypoints)
        
    except KeyboardInterrupt:
        print("\nStopping demo...")
    
    except Exception as e:
        print(f"\nError during demo: {e}")
    
    finally:
        # Cleanup
        if 'vehicle' in locals():
            vehicle.destroy()
        
        # Reset world settings
        settings = world.get_settings()
        settings.synchronous_mode = False
        world.apply_settings(settings)
        
        print("\nDemo completed.")


from figure8_track import Town04Figure8, VehicleController
main()


Analyzing track...
Roads with 8 lanes:
Road ID: 6, Lanes: [-4, -3, -2, -1, 3, 4, 5, 6]
Road ID: 35, Lanes: [-4, -3, -2, -1, 3, 4, 5, 6]
Road ID: 36, Lanes: [-4, -3, -2, -1, 3, 4, 5, 6]
Road ID: 37, Lanes: [-4, -3, -2, -1, 3, 4, 5, 6]
Road ID: 38, Lanes: [-4, -3, -2, -1, 3, 4, 5, 6]
Road ID: 39, Lanes: [-4, -3, -2, -1, 3, 4, 5, 6]
Road ID: 40, Lanes: [-4, -3, -2, -1, 3, 4, 5, 6]
Road ID: 41, Lanes: [-4, -3, -2, -1, 3, 4, 5, 6]
Road ID: 42, Lanes: [-4, -3, -2, -1, 3, 4, 5, 6]
Road ID: 43, Lanes: [-4, -3, -2, -1, 3, 4, 5, 6]
Road ID: 45, Lanes: [-4, -3, -2, -1, 3, 4, 5, 6]
Road ID: 46, Lanes: [-4, -3, -2, -1, 3, 4, 5, 6]
Road ID: 47, Lanes: [-4, -3, -2, -1, 3, 4, 5, 6]
Road ID: 48, Lanes: [-4, -3, -2, -1, 3, 4, 5, 6]
Road ID: 49, Lanes: [-4, -3, -2, -1, 3, 4, 5, 6]
Road ID: 50, Lanes: [-4, -3, -2, -1, 3, 4, 5, 6]

Validating waypoints...
Road 6: 32 waypoints
Road length: 62.1m
Road 35: 143 waypoints
Road length: 225.2m
Road 36: 41 waypoints
Road length: 80.0m
Road 37: 21 waypoints
Road l

: 